In [1]:
#coding:utf-8
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pyocr
import pyocr.builders

# matplotlibをJupyterでインライン表示するための宣言
%matplotlib inline

In [12]:
# PyOCRのセットアップ
tools = pyocr.get_available_tools() # OCRツールの有無の確認
if len(tools) == 0:
    print("No OCR tool found")

print(f"pyocr tools count = {len(tools)}")
for tool_id in range(len(tools)):
    print(f"tool[{tool_id}].tool.get_name() = " + tools[tool_id].get_name())

pyocr tools count = 2
tool[0].tool.get_name() = Tesseract (sh)
tool[1].tool.get_name() = Tesseract (C-API)


In [9]:
pyocr_tool = tools[1]
print("Will use tool '%s'" % (tool.get_name())) # 使用するOCRツールの名前が出ます，変えたい場合は一個前の参照先を変えること

langs = tool.get_available_languages() # 使用できる言語の確認
print("Available languages: %s" % ", ".join(langs))

lang = langs[0]
print("Will use lang '%s'" % (lang)) # 使用する言語について    

<module 'pyocr.libtesseract' from '/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyocr/libtesseract/__init__.py'>

In [10]:
tools[0]

<module 'pyocr.tesseract' from '/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyocr/tesseract.py'>